<a href="https://colab.research.google.com/github/sololzano/2021-Python-Optimization-Lab/blob/main/W7_Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm

In [1]:
# Import libraries
from matplotlib import pyplot as plt
import numpy as np
import time

In [2]:
# Six Hump Camel Function
def binary_camel(x, encoding, x_min):
  # From binary encoding to real number
  new_x = np.array([[int(c, 2) for c in y] for y in x])
  X = (new_x * encoding) + x_min
  x1, x2 = X[:, 0], X[:, 1]
  
  # Same
  a = (4 - 2.1*x1**2 + (1/3)*x1**4)*x1**2
  b = x1*x2
  c = (-4 + 4*x2**2)*x2**2
  return a + b +c

In [3]:
# Adjiman's Benchmark Function 
def adjiman(x, encoding, x_min):
  # From binary encoding to real number
  new_x = np.array([[int(c, 2) for c in y] for y in x])
  X = (new_x * encoding) + x_min
  x1, x2 = X[:, 0], X[:, 1]
  a = np.cos(x1) * np.sin(x2)
  b = x1 / (x2**2 + 1)
  return (a - b)

In [4]:
def init_population(fitness_func, pop_size, dimensions, precision, x_min, x_max):
  x_decimal = np.random.randint(0, (2**precision) - 1, 
                                (pop_size, dimensions))
  # From base 10 to binary
  x_binary = [[format(j, '0{}b'.format(precision)) for j in i] for i in x_decimal]
  
  encoding = (x_max - x_min)/((2**precision) - 1)

  # Calculate fitness
  fitness = fitness_func(x_binary, encoding, x_min)

  # Get best 
  min_idx = np.argmin(fitness)
  gb = x_binary[min_idx]
  fgb = fitness[min_idx]

  return x_binary, fitness, gb, fgb, encoding

In [5]:
# fitness_func, pop_size, dimensions, precision, x_min, x_max
x_binary, fitness, gb, fgb, encoding = init_population(binary_camel, 5, 2, 10, -3, 3)
x_binary

[['1101111011', '1111111110'],
 ['0001010100', '1111010011'],
 ['1101100101', '1101010001'],
 ['1110110011', '0010001100'],
 ['0001110100', '1011110011']]

In [ ]:
def selection_mechanism(fitness, kind):
  if kind == 'random':
    indices = np.random.permutation(len(fitness))[:2]
    return indices
  if kind == 'tournament':
    indices = np.random.permutation(len(fitness))[:4]
    f_indices = fitness[indices]
    idx1 = np.argmin(f_indices[0:2])
    idx2 = np.argmin(f_indices[2:]) + 2
    indices = [indices[idx1], indices[idx2]]
    return np.array(indices)
  if kind == 'roulette':
    pass

In [ ]:
indices = selection_mechanism(fitness, 'tournament')
print(indices)
print(x_binary[indices[0]])
print(x_binary[indices[1]])

[2 4]
['1100010110', '1001101100']
['1100111000', '1011000100']


In [ ]:
def crossover(parents, parents_fitness, pc, kind):
  if kind == 'single_point':
    pass
  if kind == 'double_point':
    pass
  return parents, parents_fitness

In [ ]:
def mutation(children, children_fitness, pm, kind):
  if kind == 'bit_flip':
    pass
  if kind == 'substring_swap':
    pass
  return children, children_fitness

In [ ]:
def maintenance_mechanism(parents, children, parents_fitness, 
                          children_fitness, kind):
  if kind == 'replacement':
    return 0
  if kind == 'fittest':
    return 0
  if kind == 'tournament':
    return 0

In [ ]:
def genetic_algorithm(fitness_function, pop_size, dimensions, precision, 
                      x_min, x_max, max_generations, selection_kind, 
                      crossover_kind, mutation_kind, maintenance_kind, pc, pm):
  
  # Initial population
  x, fx, gb, encoding = init_population(pop_size, dimensions, 
                                        precision, x_min, x_max)
  gb_array = []
  fb_array = []
  gb_array.append(gb)
  for i in range(1, max_generations + 1):
    new_generation = []
    new_fitness = []
    for j in range(pop_size // 2):
      # Get two potential parents
      idx_parents = selection_mechanism(fx, selection_kind)
      parents = np.copy(x[idx_parents])
      parents_fitness = np.copy(f[idx_parents])

      # Cross parents to generate two children
      children, ch_fit = crossover(parents, parents_fitness pc, crossover_kind)

      # Mutate children
      children, ch_fit = mutation(children, ch_fit, pm, mutation_kind)
      new_generation.append(np.array(children))
      new_fitness.append(np.array(ch_fit))

    # Re-shape new generation and respective fitness
    new_generation = np.reshape(new_generation, x.shape)
    new_fitness = np.reshape(new_fitness, fx.shape)

    # Maintenance mechanism
    x, fx = maintenance_mechanism(parents, new_generation, fx, 
                                  new_fitness, maintenance_kind)
    min_idx = np.argmin(fx)
    gb_array.append(x[min_idx])
    fb_array.append(fx[min_idx])
  return gb_array, fb_array